# Retrieval Tests — Supabase pgvector + OpenAI embeddings

This notebook walks through:
1. Loading environment variables (from a local `.env`).
2. Connecting to Supabase (service role key) and OpenAI.
3. Extracting text from PDFs.
4. Chunking text, generating embeddings, and uploading to Supabase.
5. Running similarity queries via the `match_documents` RPC.

**Important:**  
- Do **NOT** store your real keys in this notebook. Keep them in a local `.env` file in the project root (listed in `.gitignore`).  
- Run this notebook **from your project root** (the folder that contains `backend/`, `tools/`, `migrations/`, `pdf/`, etc.).  
- Make sure you've already run the SQL migrations in Supabase (the `migrations/` files).


In [ ]:
# Install required packages (run once).
# You may prefer to run `pip install -r requirements.txt` in your shell instead.
!pip install supabase python-dotenv pypdf openai tqdm --quiet
print('Packages install attempted (quiet mode). If this cell errors, install packages manually in your environment.')

In [ ]:
# Load environment variables from .env in the project root.
from dotenv import load_dotenv
import os, sys

# ensure we run from project root (user should start notebook from project root)
print("Current working directory:", os.getcwd())

load_dotenv()  # reads .env

SUPABASE_URL = os.environ.get("SUPABASE_URL")
SUPABASE_KEY = os.environ.get("SUPABASE_SERVICE_ROLE_KEY")
OPENAI_KEY = os.environ.get("OPENAI_API_KEY")
OPENAI_MODEL = os.environ.get("OPENAI_EMBEDDING_MODEL", "text-embedding-3-small")

print("SUPABASE_URL set?:", bool(SUPABASE_URL))
print("SUPABASE_KEY set?:", bool(SUPABASE_KEY))
print("OPENAI_KEY set?:", bool(OPENAI_KEY))
print("OPENAI_MODEL:", OPENAI_MODEL)

if not (SUPABASE_URL and SUPABASE_KEY and OPENAI_KEY):
    raise RuntimeError("Make sure your .env contains SUPABASE_URL, SUPABASE_SERVICE_ROLE_KEY, and OPENAI_API_KEY")

In [ ]:
# Create Supabase and OpenAI clients
from supabase import create_client
from openai import OpenAI

sb = create_client(SUPABASE_URL, SUPABASE_KEY)
openai = OpenAI(api_key=OPENAI_KEY)

print('Supabase and OpenAI clients created.')

In [ ]:
# PDF text extraction helper (uses pypdf)
from pypdf import PdfReader

def extract_text_from_pdf(path: str) -> str:
    reader = PdfReader(path)
    pages = []
    for page in reader.pages:
        text = page.extract_text() or ''
        pages.append(text)
    return '\n'.join(pages).strip()

# Quick local test: list pdf files if folder exists
import glob, os
pdf_files = glob.glob('pdf/*.pdf')
print('Found PDFs in pdf/:', pdf_files[:10])

In [ ]:
# Simple chunker (paragraph-aware)
def chunk_text(text: str, max_chars: int = 1500):
    paragraphs = [p.strip() for p in text.split('\n\n') if p.strip()]
    out = []
    cur = ""
    for p in paragraphs:
        if len(cur) + len(p) + 2 <= max_chars:
            cur = (cur + "\n\n" + p).strip()
        else:
            if cur: out.append(cur)
            cur = p
    if cur: out.append(cur)
    return out

# quick sanity
print('Chunker ready. Example chunk counts for first PDF (if any):')
if pdf_files:
    sample_text = extract_text_from_pdf(pdf_files[0])
    print('chars:', len(sample_text), 'chunks:', len(chunk_text(sample_text, max_chars=1500)))

In [ ]:
# Embedding helper using OpenAI client
def embed_text(text: str, model: str = OPENAI_MODEL):
    # OpenAI client returns embeddings via .embeddings.create()
    res = openai.embeddings.create(model=model, input=text)
    emb = res.data[0].embedding
    return emb

# Quick sanity: do NOT call on long texts here automatically.
print('Embedding helper ready (will call OpenAI when used).')

In [ ]:
# Upload chunks to Supabase table 'documents'
import time
from tqdm import tqdm

def upload_chunks_to_supabase(chunks, source='notebook_upload', pause_sec=0.2, max_batch=50):
    '''
    Inserts chunks into Supabase documents table.
    - chunks: list of text chunks
    - source: string identifying the source
    - pause_sec: short pause between batches to avoid rate limits
    - max_batch: number of rows per insert
    '''
    rows = []
    inserted = 0
    for i, chunk in enumerate(tqdm(chunks, desc='embedding chunks')):
        emb = embed_text(chunk)
        rows.append({
            'source': source,
            'content': chunk,
            'chunk_index': i,
            'embedding': emb
        })
        # insert in batches
        if len(rows) >= max_batch:
            res = sb.table('documents').insert(rows).execute()
            if getattr(res, 'error', None):
                print('Insert error:', res.error)
            else:
                inserted += len(rows)
            rows = []
            time.sleep(pause_sec)
    # insert remaining
    if rows:
        res = sb.table('documents').insert(rows).execute()
        if getattr(res, 'error', None):
            print('Insert error:', res.error)
        else:
            inserted += len(rows)
    return inserted

print('Upload function ready (will write to your Supabase documents table).')

In [ ]:
# Batch-upload all PDFs from pdf/ folder (run this cell to process all PDFs)
import glob, os

def process_all_pdfs(pdf_dir='pdf', chunk_size=1500):
    files = glob.glob(os.path.join(pdf_dir, '*.pdf'))
    if not files:
        print('No PDF files found in', pdf_dir)
        return
    total_inserted = 0
    for f in files:
        print('\nProcessing', f)
        txt = extract_text_from_pdf(f)
        if not txt:
            print('  - no extractable text (maybe scanned PDF). Skipping.')
            continue
        chunks = chunk_text(txt, max_chars=chunk_size)
        print(f'  - chunks: {len(chunks)}')
        inserted = upload_chunks_to_supabase(chunks, source=os.path.basename(f))
        print(f'  - inserted {inserted}')
        total_inserted += inserted
    print('\nDone. Total inserted:', total_inserted)

# To run: uncomment the next line and execute the cell
# process_all_pdfs('pdf', chunk_size=1500)
print('To run batch upload, call process_all_pdfs("pdf").')

In [ ]:
# Quick check: fetch some rows from documents
print('Fetching up to 5 documents from Supabase (for inspection):')
res = sb.table('documents').select('id, source, content, chunk_index, created_at').limit(5).execute()
if getattr(res, 'error', None):
    print('Error fetching rows:', res.error)
else:
    rows = getattr(res, 'data', [])
    for r in rows:
        print('\nID:', r.get('id'), 'source:', r.get('source'), 'chunk_index:', r.get('chunk_index'))
        print('snippet:', (r.get('content') or '')[:200].replace('\n', ' '))

In [ ]:
# Retrieval test: embed a query and call the match_documents RPC
def retrieve(query: str, k: int = 5):
    q_emb = embed_text(query)
    res = sb.rpc('match_documents', {'query_embedding': q_emb, 'match_limit': k}).execute()
    if getattr(res, 'error', None):
        raise Exception(res.error)
    return res.data

# Example query - change to something relevant to your PDFs
test_query = 'How to calm down during a panic attack?'
print('Query:', test_query)
hits = retrieve(test_query, k=5)
print('\nTop hits:')
for h in hits:
    print('distance:', h.get('distance'), 'source:', h.get('source'))
    print('snippet:', (h.get('content') or '')[:300].replace('\n',' '), '\n---')

## Troubleshooting & Notes

- **No text extracted from PDF**: many scanned PDFs are images — use OCR (Tesseract) or get the text source.  
- **Dimension mismatch errors on insert**: ensure your OpenAI embedding model returns a vector length matching `vector(1536)` in your DB. If you used a different model, change the SQL migration and function accordingly.  
- **Rate limits / slow uploads**: the notebook uses a tiny pause and batch inserts; increase `pause_sec` or reduce parallel requests if you hit OpenAI rate limits.  
- **Running the notebook**: from your project root run `jupyter notebook` or `jupyter lab`, open `retrieval_tests.ipynb`, and run cells in order.  
